# __Diffusion Kurtosis Imaging (DKI) analysis notebook__
#### __Last updated on:__ 27/02/2020
#### __Author:__ Rakshit Dadarwal

### __Requirements:__
#### 1. Multi-shell DWI dataset (minimum two bvalues)
#### 2. DIPY (https://dipy.org/)

### __This script includes:__
#### 1. DKI parametric maps estimation

In [ ]:
# import important libraries 
import os
import nibabel as nib
import timeit; timeit.timeit()

from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.reconst.dti import TensorModel

#--------------------------------------------------
# Load pre-processed DWI data and brain mask 
#--------------------------------------------------
data_path = "/home/User/Diffusion"
dwi_file = 'eddy_corrected.nii.gz'
brainmask_file = 'brain_mask.nii.gz'
bval = 'dMRI.bvals'
bvec = 'eddy_corrected.eddy_rotated_bvecs'

if os.path.exists(os.path.join(data_path,dwi_file)):
        
    img1 = nib.load(os.path.join(data_path,dwi_file))
    data = img1.get_data()

    img2 = nib.load(os.path.join(data_path,brainmask_file))
    brainmask = img2.get_data()

    bvals, bvecs = read_bvals_bvecs(os.path.join(bval),
                                    os.path.join(data_path,bvec))
    gtab = gradient_table(bvals, bvecs)

    #--------------------------------------------------------------
    #               Fit diffusion kurtosis model 
    #--------------------------------------------------------------
    print('Fitting diffuison kurtosis model')
    import dipy.reconst.dki as dki
    dkimodel = dki.DiffusionKurtosisModel(gtab)
    dkifit = dkimodel.fit(data, brainmask)
        
    #--------------------------------------------------------------
    #               Save DKI parametric maps
    #--------------------------------------------------------------
    if not os.path.exists(data_path+'/DKI/'):
        os.mkdir(data_path+'/DKI')
    data_path_saveImage = data_path+'/DKI/'
        
    FA = dkifit.fa
    MD = dkifit.md
    RD = dkifit.rd
    AD = dkifit.ad

    MK = dkifit.mk(0, 3)
    AK = dkifit.ak(0, 3)
    RK = dkifit.rk(0, 3)
        
    nib.save(nib.Nifti1Image(FA, img1.affine), os.path.join(data_path_saveImage,'dki_FA.nii.gz'))
    nib.save(nib.Nifti1Image(MD, img1.affine), os.path.join(data_path_saveImage,'dki_MD.nii.gz'))
    nib.save(nib.Nifti1Image(RD, img1.affine), os.path.join(data_path_saveImage,'dki_RD.nii.gz'))
    nib.save(nib.Nifti1Image(AD, img1.affine), os.path.join(data_path_saveImage,'dki_AD.nii.gz'))
        
    nib.save(nib.Nifti1Image(AK, img1.affine), os.path.join(data_path_saveImage,'AK.nii.gz'))
    nib.save(nib.Nifti1Image(RK, img1.affine), os.path.join(data_path_saveImage,'RK.nii.gz'))
    nib.save(nib.Nifti1Image(MK, img1.affine), os.path.join(data_path_saveImage,'MK.nii.gz'))
        
    print('Elapsed time:',timeit.timeit())
    print('Enjoy!!')